# 读取WAM5500数据文件


In [1]:
import pandas as pd
import os
import numpy as np

指定文件路径

In [2]:
fpath="data"
fname="WCSD0006.csv"
data_filename=os.path.join(fpath,fname)

In [5]:
def read_raw_data(filename):
    # csv文件中, 首行数据记录了机器型号, , 测量日期, 测量起始时间, 此信息略去未读取
    # 原始数据从下一行开始
    raw_data=pd.read_csv(filename,header=None,skiprows=1,index_col=False)
    # 数据列分别表示记录时间, 记录眼别, 视标位置, 屈光, 瞳孔直径
    raw_data.columns=["time","eye","target","power","pupil"]
    # 其中眼别和视标位置看起来对学习不重要, 故略去
    raw_data=raw_data[["time","power","pupil"]]
    
    # 测量时间使用pandas datetime格式记录, 容易使用时间间隔进行数据提取, 日期不重要
    raw_data["time"]=pd.to_datetime(raw_data["time"],unit='s')
    
    # 其他数据以float方式记录, 空缺数据记录为NaN
    raw_data["power"]=pd.to_numeric(raw_data["power"].str.strip())
    raw_data["pupil"]=pd.to_numeric(raw_data["pupil"].str.strip())
    
    # 丢弃空缺数据所在行
    raw_data.dropna(inplace=True)
    return raw_data

In [6]:
# 测试
if __name__=="__main__":
    raw_data=read_raw_data(data_filename)
    print(raw_data.head())

                     time  power  pupil
0 1970-01-01 00:00:00.000  -4.74    3.3
1 1970-01-01 00:00:00.560  -4.21    3.3
2 1970-01-01 00:00:00.710  -4.63    3.2
3 1970-01-01 00:00:00.880  -4.68    3.2
4 1970-01-01 00:00:01.060  -4.21    3.2


# 读取分类文件

分类文件是一个excel文件, 记录了分类和文件名

In [7]:
class_fname="class.xlsx"
class_filename=os.path.join(fpath,class_fname)

In [10]:
class_data=pd.read_excel(class_filename)
class_data.head()

,class,filename
0,1,WCSD0006.csv
1,1,WCSD0007.csv
2,0,WCSD0008.csv
3,0,WCSD0009.csv
